# White Rose Experiment

Aim of the experiment: prove that symbols are influenced by their color.

Case study: symbolic meanings of a white rose.

## Files needed for the experiment

HyperReal Knowledge graph (full version)

## Libraries needed

*   rdflib
*   pandas
*   numpy
*   matplotlib

## Acknowledgements

The script that generate the graph is a modified version of the script described [here](https://www.python-graph-gallery.com/circular-barplot-basic)

In [ ]:
pip install rdflib

In [ ]:
from rdflib import Namespace, URIRef
import rdflib
import rdfextras
rdfextras.registerplugins() # so we can Graph.query()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

In [ ]:
g=rdflib.ConjunctiveGraph()
g.parse("hyperreal.ttl", format="turtle")

In [ ]:
onturi = "https://w3id.org/simulation/ontology/"
dizuri = "https://w3id.org/simulation/data/"
ont = Namespace("https://w3id.org/simulation/ontology/")
diz = Namespace(dizuri)
g.bind("sim", ont)
g.bind("kb", diz)
roses = ["whiteRose"]
meanings = set()
for rose in roses:
    url = URIRef("https://w3id.org/simulation/data/"+rose)
    for s, p, o in g.triples((None, ont.hasSimulacrum, url)):
        for s2, p2, o2 in g.triples((s, ont.hasRealityCounterpart, None)):
            meanings.add(o2)

In [ ]:
rcrel = ["hasRealityCounterpart", "elicitedRealityCounterpart", "restoredRealityCounterpart", "preventedRealityCounterpart", "easedRealityCounterpart", "healedRealityCounterpart"]
pattern2 = re.compile("(?i)^red| red|blue|purple|^black | black|burgundy|white|^gold | gold|golden|^green |green |^green")

In [ ]:
firstmatch = set()
matchedcolored = dict()
for s, p, o in g.triples((None, None, ont.Simulacrum)):
    for s2, p2, o2 in g.triples((s, RDFS.label, None)):
        match = re.search(pattern2, o2)
        if match and "rose" not in o2:
            firstmatch.add(s)
            for s3, p3, o3 in g.triples((s, ont.isSimulacrumOf, None)):
                for rc in rcrel:
                    rcuri = URIRef("https://w3id.org/simulation/ontology/"+rc)
                    for s4, p4, o4 in g.triples((o3, rcuri, None)):
                        if o4 in meanings:
                            scool = s.n3()[1:-1].replace("https://w3id.org/simulation/data/", "")
                            o4cool = o4.n3()[1:-1].replace("https://w3id.org/simulation/data/", "")
                            if scool in matchedcolored:
                                matchedcolored[scool].add(o4cool)
                            else:
                                matchedcolored[scool] = set()
                                matchedcolored[scool].add(o4cool)

In [ ]:
query='''
PREFIX kb: <https://w3id.org/simulation/data/> 
PREFIX sim: <https://w3id.org/simulation/ontology/> 

SELECT ?sim ?type ?rc ?context WHERE { VALUES ?sim {kb:whiteRose}
?sim sim:isSimulacrumOf ?simulation .
?simulation a ?type ;
sim:hasRealityCounterpart|sim:restoredRealityCounterpart|sim:elicitedRealityCounterpart|sim:preventedRealityCounterpart|sim:easedRealityCounterpart|sim:healedRealityCounterpart ?rc;
sim:hasContext ?context
}
'''
zero = dict()
results = g.query(query)
for row in results:
    label = row[0].n3()[1:-1].replace("https://w3id.org/simulation/data/", "")
    rc = row[2].n3()[1:-1].replace("https://w3id.org/simulation/data/", "")
    if label in zero:
        zero[label][rc] = {"blue":0, "white":0, "gold":0, "purple":0, "red":0, "black":0, "green":0}
    else:
        if "golden" in label:
            label = label.replace("golden", "gold")
            if label in zero:
                zero[label][rc] = {"blue":0, "white":0, "gold":0, "purple":0, "red":0, "black":0, "green":0}
        else:
            zero[label] = {rc:{"blue":0, "white":0, "gold":0, "purple":0, "red":0, "black":0, "green":0}}

In [ ]:
colors = ["blue", "gold", "red", "white", "black", "purple", "green"]
for term in matchedcolored:
    for color in colors:
        if color in term:
            for meaning in matchedcolored[term]:
                for rose in zero:
                    if meaning in zero[rose]:
                        zero[rose][meaning][color] += 1

In [ ]:
one = dict()
for z in zero:
    one[z] = dict()
    for meaning in zero[z]:
        tot = 0
        for color in zero[z][meaning]:
            tot += zero[z][meaning][color]
        if tot > 0:
            one[z][meaning] = zero[z][meaning]

In [ ]:
maxm = 0
for o in one:
    for meaning in one[o]:
        possible = 0
        for color in one[o][meaning]:
            possible += one[o][meaning][color]
        if possible > maxm:
            maxm = possible
        if possible == 13:
            print(meaning)
print(maxm)

In [ ]:
rosecolor = "white"
df = pd.DataFrame(
        {
            'Name': [key for key in one[rosecolor + "Rose"]],
            'redValue': [k["red"] for k in one[rosecolor + "Rose"].values()],
            'blueValue': [k["blue"] for k in one[rosecolor + "Rose"].values()],
            'whiteValue': [k["white"] for k in one[rosecolor + "Rose"].values()],
            'blackValue': [k["black"] for k in one[rosecolor + "Rose"].values()],
            'purpleValue': [k["purple"] for k in one[rosecolor + "Rose"].values()],
            'goldValue': [k["gold"] for k in one[rosecolor + "Rose"].values()],
            'greenValue': [k["green"] for k in one[rosecolor + "Rose"].values()]
        })

In [ ]:

plt.figure(figsize=(20,10))
ax = plt.subplot(111, polar=True)
plt.axis('off')
upperLimit = 50
lowerLimit = 5
max = 15

# Let's compute heights: they are a conversion of each item value in those new coordinates
# In our example, 0 in the dataset will be converted to the lowerLimit (10)
# The maximum will be converted to the upperLimit (100)
#slope = (max - lowerLimit) / max
redheights = df.redValue
blueheights = df.blueValue
whiteheights = df.whiteValue
blackheights = df.blackValue
purpleheights = df.purpleValue
goldheights = df.goldValue
greenheights = df.greenValue

# Compute the width of each bar. In total we have 2*Pi = 360°
width = 2*np.pi / len(df.index)

# Compute the angle each bar is centered on:
indexes = list(range(1, len(df.index)+1))
angles = [element * width for element in indexes]
angles
# Draw bars
bars4= ax.bar(
    x=angles, 
    height=blackheights, 
    width=width, 
    bottom=lowerLimit +  + whiteheights + redheights + goldheights + greenheights + blueheights + purpleheights,
    linewidth=0, 
    edgecolor="black",
    color="#000000",
)
bars = ax.bar(
    x=angles, 
    height=redheights, 
    width=width, 
    bottom=lowerLimit + whiteheights,
    linewidth=0, 
    edgecolor="black",
    color="#e00700",
)
bars6= ax.bar(
    x=angles, 
    height=goldheights, 
    width=width, 
    bottom=lowerLimit + whiteheights + redheights,
    linewidth=0, 
    edgecolor="black",
    color="#ffc400",
)
bars7= ax.bar(
    x=angles, 
    height=greenheights, 
    width=width, 
    bottom=lowerLimit + whiteheights + redheights + goldheights,
    linewidth=0, 
    edgecolor="black",
    color="green",
)

bars2= ax.bar(
    x=angles, 
    height=blueheights, 
    width=width, 
    bottom=lowerLimit + whiteheights + redheights + goldheights + greenheights,
    linewidth=0, 
    edgecolor="black",
    color="#0033ff",
)
bars5= ax.bar(
    x=angles, 
    height=purpleheights, 
    width=width, 
    bottom=lowerLimit + whiteheights + redheights + goldheights + greenheights + blueheights,
    linewidth=0, 
    edgecolor="black",
    color="#ea00ff",
)
bars3= ax.bar(
    x=angles, 
    height=whiteheights, 
    width=width, 
    bottom=lowerLimit,
    linewidth=0, 
    edgecolor="black",
    color="#f0f2f2",
)



# little space between the bar and the label
labelPadding = 1

# Add labels
for bar, bar1, bar2, bar3, bar4, bar5, bar6, angle, height, label in zip(bars, bars2, bars3, bars4, bars5, bars6, bars7, angles, goldheights, df["Name"]):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y=lowerLimit + labelPadding + bar.get_height() + bar1.get_height() + bar2.get_height() + bar3.get_height() + bar4.get_height() + bar5.get_height() + bar6.get_height(), 
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor")

#plt.savefig(rosecolor+'.png')
#plt.savefig(rosecolor+'.pdf')

#Uncomment the two commented lines above to save the generated graph as either
# a png or pdf file